### 案例
    由于Python中全局解释器锁(GIL)的存在,在任意时刻只允许一个线程在解释器中运行,因此Python的多线程不适合处理cpu密集性的任务
    
    想要处理cpu密集性的任务,可以使用多进程模型

### 解决方案
    使用标准库中multiprocessing.Process，它可以启动子线程进程执行任务,操作接口，进程间通信,进程键同步等都与Threading.Thread类似

In [1]:
from multiprocessing import Process

In [2]:
def f(s):
    print s

In [4]:
p = Process(target=f, args=('hello', ))

In [5]:
p.start()

hello


In [6]:
p.join()

#### 多个进程中，之间的的地址空间是相互独立的

In [7]:
x = 1

In [8]:
def f():
    global x
    x = 5

In [9]:
f()

In [10]:
x

5

In [11]:
x = 1

In [12]:
p = Process(target=f)

In [13]:
p.start()

In [15]:
x #---->子进程和主进程看到的x是相互独立的

1

### 进程之间如何进行通信

In [16]:
from multiprocessing import Queue, Pipe #进程环境下的Queue

In [17]:
q = Queue()

In [18]:
q.put(1)

In [19]:
q.get()

1

In [20]:
def f(q):
    print 'start'
    print q.get()
    print 'end'

In [21]:
Process(target=f,args=(q, )).start()

start
100
end


In [22]:
q.put(100)

In [23]:
c1, c2 = Pipe() #会创建双向的管道

In [24]:
c1.send('abc') #在c1管道中传入abc

In [25]:
c2.recv() #在c2中取出来

'abc'

In [26]:
c2.send('xyz')

In [27]:
c1.recv()

'xyz'

In [28]:
def f(c):
    c.send(c.recv() * 2)

In [29]:
c1, c2 = Pipe()

In [30]:
Process(target=f, args=(c2, )).start()

In [31]:
c1.send(55)

In [32]:
c1.recv()

110

### 案例代码 -> 用多进程和多线程寻找水仙花数

In [5]:
from threading import Thread
from multiprocessing import Process

def isArmstrong(n):
    a, t = [], n
    while t > 0:
        a.append(t % 10)
        t /= 10
        
    k = len(a)
    return sum (x ** k for x in a) == n

def findArmstrong(a, b):
    print a, b
    res = [k for k in range(a, b) if isArmstrong(k)]
    print '%s ~ %s: %s' % (a, b, res)
    
def findByThread(*argslist):
    workers = []
    for args in argslist:
        worker = Thread(target=findArmstrong, args=args)
        workers.append(worker)
        worker.start()
        
    for worker in workers:
        worker.join()
        
def findByProcess(*argslist):
    workers = []
    for args in argslist:
        worker = Process(target=findArmstrong, args=args)
        workers.append(worker)
        worker.start()
        
    for worker in workers:
        worker.join()
        
if __name__ == '__main__':
    import time
    start = time.time()
#     findByProcess((20000000, 25000000), (25000000, 30000000), (35000000, 40000000), (45000000, 50000000)) # 18.4651420116
    findByThread((20000000, 25000000), (25000000, 30000000), (35000000, 40000000), (45000000, 50000000))  # 49.2440731525
    print time.time() - start
        

20000000 25000000
25000000 35000000 40000000
30000000
45000000 50000000
25000000 ~ 30000000: []
45000000 ~ 50000000: []
35000000 ~ 40000000: []
20000000 ~ 25000000: [24678050, 24678051]
128.61290288
